## 第一步 定义总体

In [1]:
from zipline.pipeline.data import CNEquityPricing
from zipline.pipeline.fundamentals import Fundamentals

In [2]:
market_cap = CNEquityPricing.market_cap.latest

In [3]:
# 市值前1000位
universe = market_cap.top(1000)

## 第二步 定义因子

In [4]:
from zipline.pipeline.factors import SimpleMovingAverage

In [5]:
# 1-month moving average factor
fast_ma = SimpleMovingAverage(inputs=[CNEquityPricing.close],window_length=21)
# 6 month
slow_ma = SimpleMovingAverage(inputs=[CNEquityPricing.close], window_length=126)
momentum = fast_ma / slow_ma
momentum_factor = momentum.zscore()

### 运行pipeline

In [6]:
from zipline.pipeline import Pipeline
from zipline.pipeline.domain import CN_EQUITIES
from zipline.research import run_pipeline

In [7]:
pipe = Pipeline(
    columns = {
        'momentum_factor':momentum_factor
    },
    screen=universe,
    domain=CN_EQUITIES
)

In [8]:
factor_data = run_pipeline(pipe, '2016-01-01', '2019-01-01')

In [9]:
factor_data.tail()

momentum_factor
datetime                  asset                        
2018-12-28 00:00:00+00:00 晨光文具(603899)         0.384056
                          益丰药房(603939)        -0.501404
                          兆易创新(603986)        -1.598242
                          艾华集团(603989)         0.145367
                          洛阳钼业(603993)        -0.925125

## 测试因子
使用`alphalens`检验因子的预测能力。

In [10]:
from zipline.pipeline.factors import Returns

In [11]:
returns_pipe = Pipeline(
    columns={
        '1D':Returns(window_length=2)
    },
    domain=CN_EQUITIES
)
returns_data = run_pipeline(returns_pipe, '2016-01-01','2019-01-01')

In [12]:
import alphalens as al
import pandas as pd

In [13]:
from zipline.research import get_forward_returns

In [14]:
# Get the 1-day forward returns for the assets and dates in the factor
returns_df = get_forward_returns(
    factor_data['momentum_factor'],
    [1],
    CN_EQUITIES
)

# Format the factor and returns data so that we can run it through Alphalens.
al_data = al.utils.get_clean_factor(
    factor_data['momentum_factor'],
    returns_df,
    quantiles=5,
    bins=None,
)

Dropped 0.1% entries from factor data: 0.1% in forward returns computation and 0.0% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).
max_loss is 35.0%, not exceeded: OK!


Then, we can create a factor tearsheet to analyze our momentum factor.

In [15]:
from alphalens.tears import create_full_tear_sheet

create_full_tear_sheet(al_data)

The Alphalens tearsheet offers insight into the predictive ability of a factor. To learn more about Alphalens, check out the documentation.

# Step 4 - Discuss with the Quantopian Community

When we have a factor that we like, we can share the result in the Quantopian community forum and solicit feedback from community members. The ideas you come up with on Quantopian belong to you, but sometimes sharing a result can spark a discussion and create an opportunity to learn from others.
In the community forum, Research notebooks can be attached to posts. If you want to share the result of your work and the code, you can share your notebook as is. If you want to keep the code a secret, you can create a copy of your notebook, run your factor through Alphalens, delete the code cells that have your Pipeline code, and just share the Alphalens result in a community post.
If you want to share your work or your result in the community, adding an explanation or asking questions will make it more likely to have a productive conversation with other users.

# Recap and Next Steps

In this tutorial, we introduced Quantopian and walked through a factor research workflow using Pipeline and Alphalens. Quantopian has a rich set of documentation which you can use to learn more about the platform. We recommend starting with the User Guide section of the documentation if you would like to grow your understanding of Quantopian or the Data Reference if you want to learn more about the data that's available to you out of the box.